In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import lightgbm as lgb

class UJIDataset(torch.utils.data.Dataset):
    def __init__(self, root, train = True, transform = None, target_transform = None, download = False):
        self.root = root
        dir_path = self.root + '/UJIndoorLoc'
        zip_path = self.root + '/uji_uil.zip'
        dataset_training_file = dir_path + '/trainingData.csv'
        dataset_validation_file = dir_path + '/validationData.csv'
        # Load independent variables (WAPs values)
        if train:
            dataset_file = dataset_training_file
        else:
            dataset_file = dataset_validation_file
        file = open(dataset_file, 'r')
        # Load labels
        label = file.readline()
        label = label.split(',')
        # Load independent variables
        file_load = np.loadtxt(file, delimiter = ',', skiprows = 1)
        #file_load_label = np.loadtxt(file, delimiter = ',')
        #data = np.genfromtxt(file, dtype = float, delimiter = ',', names = True)
        # RSSI values
        self.x = file_load[:, 0 : 520]
        # Load dependent variables
        self.y = file_load[:, 520 : 524]
        # Divide labels into x and y
        self.x_label = label[0 : 520]
        self.x_label = np.concatenate([self.x_label, label[524: 529]])
        self.y_label = label[520 : 524]
        # Regularization of independent variables
        self.x[self.x == 100] = np.nan    # WAP not detected
        self.x = self.x + 104             # Convert into positive values
        self.x = self.x / 104             # Regularize into scale between 0 and 1
        # Building ID, Space ID, Relative Position, User ID, Phone ID and Timestamp respectively
        self.x = np.concatenate([self.x, file_load[:, 524 : 529]], axis = 1)
        file.close()
        # Reduce the number of dependent variables by combining building number and floor into one variable: area
        area = self.y[:, 3] * 5 + self.y[:, 2]
        self.y = np.column_stack((self.y, area))
    def to_tensor(self):
        self.x = torch.from_numpy(self.x).float()
        self.y = torch.from_numpy(self.y).float()
        self.area = torch.from_numpy(self.area).float()
    def nan_to_zero(self):
        self.x = np.nan_to_num(self.x)
    # Return the target instance (row)
    def __getitem__(self, index_row):
        return self.x[index_row, :], self.y[index_row, :]
    # Return the number of instances (the number of rows)
    def __len__(self, dim = 0):
        return int(self.x.size()[0])

In [3]:
# Calculate Euclidean distance (unit: meter) between two coordinates in EPSG:3857 
def euclidean_distance(latitude_1, longitude_1, latitude_2, longitude_2):
    return np.sqrt((latitude_1 - latitude_2)**2 + (longitude_1 - longitude_2)**2)

Import dataset with lightgbm

In [3]:
a = torch.zeros([2,2])
print(a)
b = torch.ones([2,2])
c = torch.cat((a, b), dim = 1)
print(c)

tensor([[0., 0.],
        [0., 0.]])
tensor([[0., 0., 1., 1.],
        [0., 0., 1., 1.]])


In [4]:
# Load training dataset
dataset_train = UJIDataset('./data', train = True)
dataset_test = UJIDataset('./data', train = False)
#gb_train_data = lgb.Dataset(

In [40]:
#dataset_train.nan_to_zero()
#dataset_validate.nan_to_zero()
#dataset_test.nan_to_zero()
#dataset_train.to_tensor()
#dataset_validate.to_tensor()


In [32]:
print(dataset_train.x)

[[           nan            nan            nan ... 2.00000000e+00
  2.30000000e+01 1.37171369e+09]
 [           nan            nan            nan ... 2.00000000e+00
  2.30000000e+01 1.37171410e+09]
 [           nan            nan            nan ... 2.00000000e+00
  2.30000000e+01 1.37171381e+09]
 ...
 [           nan            nan            nan ... 1.80000000e+01
  1.00000000e+01 1.37171092e+09]
 [           nan            nan            nan ... 1.80000000e+01
  1.00000000e+01 1.37171105e+09]
 [           nan            nan            nan ... 1.80000000e+01
  1.00000000e+01 1.37171102e+09]]


In [31]:
print(dataset_test.x)

[[           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38115505e+09]
 [           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38115510e+09]
 [           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38115514e+09]
 ...
 [           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38124778e+09]
 [           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38124781e+09]
 [           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38124784e+09]]


In [26]:
x_train, x_validate, y_train, y_validate = sklearn.model_selection.train_test_split(dataset_train.x, dataset_train.y, test_size = 0.2, random_state = 42)
x_test1, x_test2, y_test1, y_test2 = sklearn.model_selection.train_test_split(dataset_test.x, dataset_test.y, test_size = 0.5, random_state = 42)

In [29]:
print(x_train)

[[           nan            nan            nan ... 1.00000000e+00
  1.40000000e+01 1.37087874e+09]
 [           nan            nan            nan ... 6.00000000e+00
  1.90000000e+01 1.37171987e+09]
 [           nan            nan            nan ... 1.00000000e+01
  8.00000000e+00 1.37172087e+09]
 ...
 [           nan            nan            nan ... 9.00000000e+00
  1.40000000e+01 1.37172008e+09]
 [           nan            nan            nan ... 2.00000000e+00
  2.30000000e+01 1.37172121e+09]
 [           nan            nan            nan ... 1.00000000e+00
  1.40000000e+01 1.37103670e+09]]


In [42]:
train_size = int(0.2 * len(dataset_train.x))
dataset_train_reg_long_lgb = lgb.Dataset(x_train, label = y_train[:, 0])
dataset_validate_reg_long_lgb = lgb.Dataset(x_validate, label = y_validate[:, 0])
dataset_train_reg_lat_lgb = lgb.Dataset(x_train, label = y_train[:, 1])
dataset_validate_reg_lat_lgb = lgb.Dataset(x_validate, label = y_validate[:, 1])
dataset_train_cat_floor_lgb = lgb.Dataset(x_train, label = y_train[:, 2])
dataset_validate_cat_floor_lgb = lgb.Dataset(x_validate, label = y_validate[:, 2])
dataset_train_cat_building_lgb = lgb.Dataset(x_train, label = y_train[:, 3])
dataset_validate_cat_building_lgb = lgb.Dataset(x_validate, label = y_validate[:, 3])
dataset_train_cat_area_lgb = lgb.Dataset(x_train, label = y_train[:, 4])
dataset_validate_cat_area_lgb = lgb.Dataset(x_validate, label = y_validate[:, 4])

In [15]:
# feval(preds: array, train_data: Dataset) -> name: string, eval_result: float, is_higher_better: bool
def multiclass_score(preds, train_data):
    label = train_data.get_label()
    return 'multiclass_score', np.mean(label == preds), True

def regression_score(preds, train_data):
    label = train_data.get_label()
    error_distance = []
    for i in range(label):
        error_distance.append(euclidean_distance(preds[i], 0, label[i], 0))
    return 'regression_score', np.mean(error_distance), False

In [43]:
params_reg = {'learning_rate': 0.001,
              'num_boost_round': 500,
              'max_depth': 10,
              'boosting': 'rf', 
              'objective': 'regression',
              #'metric': 'rmse',
              'metric': 'None',
              'is_training_metric': True, 
              'num_leaves': 1440,  
              #'feature_fraction': 0.7, 
              'bagging_fraction': 0.8, 
              'bagging_freq': 5,
              #'early_stopping_round': 100,
              #'seed':2018
             }
params_cat_building = {'learning_rate': 0.002,
              'num_boost_round': 300,
              'max_depth': 16,
              'boosting': 'rf',
              'objective': 'multiclass',
              'num_class': 3,
              #'metric': 'multi_logloss',
              'metric': 'None',
              #'is_training_metric': True,
              'num_leaves': 144,
              #'feature_fraction': 0.97,
              'bagging_fraction': 0.7,
              'bagging_freq': 5,
              #'early_stopping_round': 100,
             }
params_cat_floor = {'learning_rate': 0.002,
              'num_boost_round': 300,
              'max_depth': 16,
              'boosting': 'rf',
              'objective': 'multiclass',
              'num_class': 5,
              #'metric': 'multi_logloss',
              'metric': 'None',
              #'is_training_metric': True,
              'num_leaves': 144,
              #'feature_fraction': 0.97,
              'bagging_fraction': 0.7,
              'bagging_freq': 5,
              #'early_stopping_round': 100,
             }
params_cat_area = {'learning_rate': 0.002,
              'num_boost_round': 300,
              'max_depth': 16,
              'boosting': 'rf',
              'objective': 'multiclass',
              'num_class': 15,
              #'metric': 'multi_logloss',
              'metric': 'None',
              #'is_training_metric': True,
              'num_leaves': 144,
              #'feature_fraction': 0.97,
              'bagging_fraction': 0.7,
              'bagging_freq': 5,
              #'early_stopping_round': 100,
             }

In [44]:
model_reg_long_lgb = lgb.train(params_reg, dataset_train_reg_long_lgb, dataset_validate_reg_long_lgb, feval = regression_score, verbose_eval = 100)#, early_stopping_rounds = 100)
model_reg_lat_lgb = lgb.train(params_reg, dataset_train_reg_lat_lgb, dataset_validate_reg_lat_lgb, feval = regression_score, verbose_eval = 100)#, early_stopping_rounds = 100)
model_cat_floor_lgb = lgb.train(params_cat_floor, dataset_train_cat_floor_lgb, dataset_validate_cat_floor_lgb, feval = multiclass_score, verbose_eval = 100)#, early_stopping_rounds = 100)
model_cat_building_lgb = lgb.train(params_cat_building, dataset_train_cat_building_lgb, dataset_validate_cat_building_lgb, feval = multiclass_score, verbose_eval = 100)#, early_stopping_rounds = 100)
model_cat_area_lgb = lgb.train(params_cat_area, dataset_train_cat_area_lgb, dataset_validate_cat_area_lgb, feval = multiclass_score, verbose_eval = 100)#, early_stopping_rounds = 100)

In [ ]:
#lgbr = lgb.LGBMRegressor(**params_lgbr)
#lgbc = lgb.LGBMClassifier(**params_lgbc)
#model_reg_long_lgb_fit = lgbr.fit(X = x_train, y = y_train[:, 0], **params_lgbr_long_fit, early_stopping_rounds = 100)
#model_reg_lat_lgb_fit = lgbr.fit(X = x_train, y = y_train[:, 1], **params_lgbr_lat_fit, early_stopping_rounds = 100)
#model_cat_floor_lgb_fit = lgbc.fit(X = x_train, y = y_train[:, 2], **params_lgbc_floor_fit, early_stopping_rounds = 100)
#model_cat_building_lgb_fit = lgbc.fit(X = x_train, y = y_train[:, 3], **params_lgbc_building_fit, early_stopping_rounds = 100)
#model_cat_area_lgb_fit = lgbc.fit(X = x_train, y = y_train[:, 4], **params_lgbc_area_fit, early_stopping_rounds = 100)

In [45]:
predict_long_train = model_reg_long_lgb.predict(x_train)
predict_long_validate = model_reg_long_lgb.predict(x_validate)
predict_lat_train = model_reg_lat_lgb.predict(x_train)
predict_lat_validate = model_reg_lat_lgb.predict(x_validate)
predict_floor_train = model_cat_floor_lgb.predict(x_train)
predict_floor_validate = model_cat_floor_lgb.predict(x_validate)
predict_building_train = model_cat_building_lgb.predict(x_train)
predict_building_validate = model_cat_building_lgb.predict(x_validate)
predict_area_train = model_cat_area_lgb.predict(x_train)
predict_area_validate = model_cat_area_lgb.predict(x_validate)

In [49]:
predict_long_test1 = model_reg_long_lgb.predict(x_test1)
predict_lat_test1 = model_reg_lat_lgb.predict(x_test1)
predict_floor_test1 = model_cat_floor_lgb.predict(x_test1)
predict_building_test1 = model_cat_building_lgb.predict(x_test1)
predict_area_test1 = model_cat_area_lgb.predict(x_test1)

In [42]:
#predict_long_test1_fit = model_reg_long_lgb_fit.predict(x_test1)
#predict_lat_test1_fit = model_reg_lat_lgb_fit.predict(x_test1)
#predict_floor_test1_fit = model_cat_floor_lgb_fit.predict(x_test1)
#predict_building_test1_fit = model_cat_building_lgb_fit.predict(x_test1)
#predict_area_test1_fit = model_cat_area_lgb_fit.predict(x_test1)

In [28]:
#print(dataset_test.x)

[[           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38115505e+09]
 [           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38115510e+09]
 [           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38115514e+09]
 ...
 [           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38124778e+09]
 [           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38124781e+09]
 [           nan            nan            nan ... 0.00000000e+00
  1.30000000e+01 1.38124784e+09]]


In [29]:
print(y_train[:, 1])
print(predict_lat_train)

[4864931.4921   4864849.3028   4864842.961439 ... 4864790.0184
 4864769.4062   4865010.6595  ]
[4864827.88861666 4864848.22039905 4864832.51212763 ... 4864826.98364879
 4864768.70698926 4864827.13400571]


In [30]:
print(y_train[:, 0])
print(predict_long_train)
print(y_validate[:, 0])
print(predict_long_validate)
print(y_train[:, 2])
print(predict_floor_train.argmax(axis = 1))
print(y_validate[:, 2])
print(predict_floor_validate.argmax(axis = 1))

[-7680.1346     -7372.8029     -7377.07826999 ... -7401.9962
 -7368.3861     -7640.9762    ]
[-7410.30554739 -7370.12065268 -7367.42936381 ... -7410.12964627
 -7367.99123466 -7442.58528594]
[-7511.5215     -7535.39365217 -7348.8982     ... -7408.69525072
 -7445.55787384 -7390.7612    ]
[-7513.01743653 -7534.592519   -7349.7597729  ... -7431.72326272
 -7448.09434054 -7360.67742061]
[0. 4. 0. ... 0. 3. 0.]
[3 4 1 ... 1 3 3]
[2. 2. 3. ... 2. 0. 3.]
[2 2 3 ... 3 0 1]


In [ ]:
print(y_test1[:, 0])
print(predict_long_test1)
print(y_test1[:, 1])
print(predict_lat_test1)
print(y_test1[:, 2])
print(predict_floor_test1.argmax(axis = 1))

In [18]:
print(len(predict_floor_validate))

3988


In [19]:
print(len(predict_floor_test))

1110


In [46]:
error_building_train = []
error_building_validate = []
error_floor_train = []
error_floor_validate = []
error_area_train = []
error_area_validate = []
predict_floor_argmax_train = predict_floor_train.argmax(axis = 1)
predict_floor_argmax_validate = predict_floor_validate.argmax(axis = 1)
predict_building_argmax_train = predict_building_train.argmax(axis = 1)
predict_building_argmax_validate = predict_building_validate.argmax(axis = 1)
predict_area_argmax_train = predict_area_train.argmax(axis = 1)
predict_area_argmax_validate = predict_area_validate.argmax(axis = 1)
for i in range(len(predict_floor_train)):
    if predict_floor_argmax_train[i] != y_train[i, 2]:
        error_floor_train.append(i)
    if predict_building_argmax_train[i] != y_train[i, 3]:
        error_building_train.append(i)
    if predict_area_argmax_train[i] != y_train[i, 4]:
        error_area_train.append(i)
for i in range(len(predict_floor_validate)):
    if predict_floor_argmax_validate[i] != y_validate[i, 2]:
        error_floor_validate.append(i)
    if predict_building_argmax_validate[i] != y_validate[i, 3]:
        error_building_validate.append(i)
    if predict_area_argmax_validate[i] != y_validate[i, 4]:
        error_area_validate.append(i)

In [47]:
error_rate_floor_train = len(error_floor_train) / len(predict_floor_train)
print('error_rate_floor_train', error_rate_floor_train)
error_rate_floor_validate = len(error_floor_validate) / len(predict_floor_train)
print('error_rate_floor_validate', error_rate_floor_validate)
error_rate_building_train = len(error_building_train) / len(predict_building_train)
print('error_rate_building_train', error_rate_building_train)
error_rate_building_validate = len(error_building_validate) / len(predict_building_train)
print('error_rate_buildilng_validate', error_rate_building_validate)
error_rate_area_train = len(error_area_train) / len(predict_area_train)
print('error_rate_area_train', error_rate_area_train)
error_rate_area_validate = len(error_area_validate) / len(predict_area_validate)
print('error_rate_area_validate', error_rate_area_validate)

error_rate_floor_train 0.012666165036368197
error_rate_floor_validate 0.0036995234512164537
error_rate_building_train 0.0007524454477050414
error_rate_buildilng_validate 0.0003762227238525207
error_rate_area_train 0.021381991472284926
error_rate_area_validate 0.02958876629889669


In [22]:
print(len(predict_long_train))

15948


In [48]:
error_distance_train = []
error_distance_validate = []
for i in range(len(predict_long_train)):
    error_distance_train.append(euclidean_distance(predict_lat_train[i], predict_long_train[i], y_train[i, 1], y_train[i, 0]))
for i in range(len(predict_long_validate)):
    error_distance_validate.append(euclidean_distance(predict_lat_validate[i], predict_long_validate[i], y_validate[i, 1], y_validate[i, 0]))
error_mean_distance_train = np.mean(np.stack(error_distance_train)).item()
error_max_distance_train = np.max(np.stack(error_distance_train)).item()
error_min_distance_train = np.min(np.stack(error_distance_train)).item()
error_std_distance_train = np.std(np.stack(error_distance_train)).item()
error_var_distance_train = np.var(np.stack(error_distance_train)).item()
print('error_mean_distance_train: ', error_mean_distance_train)
print('error_max_distance_train: ', error_max_distance_train)
print('error_min_distance_train: ', error_min_distance_train)
print('error_std_distance_train: ', error_std_distance_train)
print('error_var_distance_train: ', error_var_distance_train)
error_mean_distance_validate = np.mean(np.stack(error_distance_validate)).item()
error_max_distance_validate = np.max(np.stack(error_distance_validate)).item()
error_min_distance_validate = np.min(np.stack(error_distance_validate)).item()
error_std_distance_validate = np.std(np.stack(error_distance_validate)).item()
error_var_distance_validate = np.var(np.stack(error_distance_validate)).item()
print('error_mean_distance_validate: ', error_mean_distance_validate)
print('error_max_distance_validate: ', error_max_distance_validate)
print('error_min_distance_validate: ', error_min_distance_validate)
print('error_std_distance_validate: ', error_std_distance_validate)
print('error_var_distance_validate: ', error_var_distance_validate)

error_mean_distance_train:  8.950692895133042
error_max_distance_train:  174.74817294686335
error_min_distance_train:  0.006624364926024075
error_std_distance_train:  10.924766425035246
error_var_distance_train:  119.35052144157738
error_mean_distance_validate:  9.417689133326265
error_max_distance_validate:  148.5944940024184
error_min_distance_validate:  0.10363125191523935
error_std_distance_validate:  11.20934584112626
error_var_distance_validate:  125.64943418597456


In [34]:
error_building_test1 = []
error_floor_test1 = []
error_area_test1 = []
predict_floor_argmax_test1 = predict_floor_test1.argmax(axis = 1)
predict_building_argmax_test1 = predict_building_test1.argmax(axis = 1)
predict_area_argmax_test1 = predict_area_test1.argmax(axis = 1)
for i in range(len(predict_floor_test1)):
    if predict_floor_argmax_test1[i] != y_test1[i, 2]:
        error_floor_test1.append(i)
    if predict_building_argmax_test1[i] != y_test1[i, 3]:
        error_building_test1.append(i)
    if predict_area_argmax_test1[i] != y_test1[i, 4]:
        error_area_test1.append(i)

In [50]:
error_rate_floor_test1 = len(error_floor_test1) / len(predict_floor_test1)
print('error_rate_floor_test1', error_rate_floor_test1)
error_rate_building_test1 = len(error_building_test1) / len(predict_building_test1)
print('error_rate_building_test1', error_rate_building_test1)
error_rate_area_test1 = len(error_area_test1) / len(predict_area_test1)
print('error_rate_area_test1', error_rate_area_test1)

error_rate_floor_test1 0.7027027027027027
error_rate_building_test1 0.618018018018018
error_rate_area_test1 0.8828828828828829


In [51]:
error_distance_test1 = []
for i in range(len(predict_long_test1)):
    error_distance_test1.append(euclidean_distance(predict_lat_test1[i], predict_long_test1[i], y_test1[i, 1], y_test1[i, 0]))
error_mean_distance_test1 = np.mean(np.stack(error_distance_test1)).item()
error_max_distance_test1 = np.max(np.stack(error_distance_test1)).item()
error_min_distance_test1 = np.min(np.stack(error_distance_test1)).item()
error_std_distance_test1 = np.std(np.stack(error_distance_test1)).item()
error_var_distance_test1 = np.var(np.stack(error_distance_test1)).item()
print('error_mean_distance_test1: ', error_mean_distance_test1)
print('error_max_distance_test1: ', error_max_distance_test1)
print('error_min_distance_test1: ', error_min_distance_test1)
print('error_std_distance_test1: ', error_std_distance_test1)
print('error_var_distance_test1: ', error_var_distance_test1)

error_mean_distance_test1:  132.64861067537527
error_max_distance_test1:  220.85087291266163
error_min_distance_test1:  0.19723348205642666
error_std_distance_test1:  59.841505318076
error_var_distance_test1:  3581.0057587333185
